# Machine Translation with Recurrent Neural Networks

This code is a modified version of the code found here: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [0]:
!mkdir -p local_modules/loader
!mkdir -p local_modules/project_tests
!mkdir -p data/

In [0]:
import sys
sys.path.append('local_modules/')

In [4]:
# Import the libraries
import collections
import loader
import numpy as np
import project_tests as tests
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.models import Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


In [41]:
# Run this cell only if you encounter problems importing libraries
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Load the data: English sentences + French equivalents
english_sentences = loader.load_data('data/small_vocab_en.txt')
french_sentences = loader.load_data('data/small_vocab_fr.txt')

print('Dataset Loaded')

Dataset Loaded


In [15]:
# Print example sentences 
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


In [16]:
# Data exploration
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [0]:
# Data pre-processing - Tokenizer function
def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

In [17]:
# Data pre-processing - Padding function
def pad(x, length=None):

    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [18]:
# Data pre-processing - Pipeline including padding and tokenization
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index) + 1
french_vocab_size = len(french_tokenizer.word_index) + 1

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 200
French vocabulary size: 345


In [12]:
# Logits to text function: Mapping of logits to French words
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [19]:
# A simple recurrent neural network (RNN) model
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 110288 samples, validate on 27573 samples
Epoch 1/10





110288/110288 [==============================] - 49s 442us/step - loss: 3.5049 - acc: 0.4007 - val_loss: 2.6194 - val_acc: 0.4556
Epoch 2/10
110288/110288 [==============================] - 47s 427us/step - loss: 2.5078 - acc: 0.4640 - val_loss: 2.3959 - val_acc: 0.4775
Epoch 3/10
110288/110288 [==============================] - 47s 429us/step - loss: 2.2733 - acc: 0.4962 - val_loss: 2.1417 - val_acc: 0.5223
Epoch 4/10
110288/110288 [==============================] - 47s 429us/step - loss: 2.0173 - acc: 0.5373 - val_loss: 1.8869 - val_acc: 0.5654
Epoch 5/10
110288/110288 [==============================] - 47s 427us/step - loss: 1.7973 - acc: 0.5744 - val_loss: 1.7231 - val_acc: 0.5827
Epoch 6/10
110288/110288 [==============================] - 47s 428us/step - loss: 1.6759 - acc: 0.5825 - val_loss: 1.6331 - val_acc: 0.5868
E

In [0]:
# The Encoder-Decoder model 
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False)) # Encoder
    model.add(RepeatVector(output_sequence_length)) # Context vector
    model.add(GRU(128, return_sequences = True)) # Decoder
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model


tests.test_encdec_model(encdec_model)
tmp_x = pad(preproc_english_sentences)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))

encodeco_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

encodeco_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)
print(logits_to_text(encodeco_model.predict(tmp_x[:1])[0], french_tokenizer))

/usr/local/lib/python3.6/dist-packages/keras/engine/sequential.py:111: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 99s 902us/step - loss: 2.9881 - acc: 0.4424 - val_loss: 2.4311 - val_acc: 0.4865
Epoch 2/20
110288/110288 [==============================] - 97s 882us/step - loss: 2.2779 - acc: 0.5008 - val_loss: 2.1252 - val_acc: 0.5040
Epoch 3/20
110288/110288 [==============================] - 97s 879us/step - loss: 2.0423 - acc: 0.5082 - val_loss: 1.9682 - val_acc: 0.5213
Epoch 4/20
110288/110288 [==============================] - 97s 879us/step - loss: 1.9021 - acc: 0.5281 - val_loss: 1.8249 - val_acc: 0.5404
Epoch 5/20
110288/110288 [==============================] - 97s 883us/step - loss: 1.7659 - acc: 0.5478 - val_loss: 1.7035 - val_acc: 0.5586
Epoch 6/20
110288/110288 [==============================] - 97s 877us/step - loss: 1.6635 - acc: 0.5698 - val_loss: 1.6184 - val_acc: 0.5759
Epoch 7/20
110288/110288 [==============================] - 97s 882us/step - loss: 1.5870 - acc: 0.5796